In [1]:
# 超时

class Solution:

    def beautifulPartitions(self, s: str, k: int, minLength: int) -> int:
        # dp[i][j] j个数字分成i段的方法数 i:[0,k] j:[0,n]
        # dp[i][j] = sum(dp[i-1][j-x]) s[j-x:j]符合要求
        # basecase dp[0][j] = 0 dp[0][0] = 1
        def is_prime(n):
            return n in '2357'

        def check(l, r):  # s[l,r]是否符合
            return is_prime(s[l]) and not is_prime(s[r])

        mod = 10**9 + 7
        n = len(s)
        if n < k * minLength or not is_prime(s[0]) or is_prime(s[-1]):
            return 0
        dp = [[0] * (n + 1) for _ in range(k + 1)]
        dp[0][0] = 1
        for i in range(1, k + 1):
            for j in range(n + 1):
                if j < minLength:
                    continue
                for x in range(minLength, j + 1):
                    if check(j - x, j - 1):
                        dp[i][j] += dp[i - 1][j - x]
                        dp[i][j] %= mod
        # pprint(dp)
        return dp[-1][-1] % mod


s = Solution()
s.beautifulPartitions('23542185131', 3, 2)


3

In [2]:
# 超时一点 比上一版好
'''
1. 问题中有哪些变量?
分割个数 k
字符串长度 n

2. 重新复述一遍问题,替换变量名
把一个长度为j的字符串,分割出i段的合法方案数

3. (关键)最后一步发生了什么?
分割出一个子串
长度为 x ,且这个子串是s的一个后缀

4. 去掉最后一步,问题规模缩小了,变成什么样了?
把一个长度为 j-x 的字符串,分割出 i-1 段的合法方案数

5. 得到状态转移方程
2-> dp[i][j] 把一个长度为j的字符串,分割出i段的合法方案数
4-> dp[i][j] += dp[i-1][j-x] j-x-1是子问题你结尾的索引
    j>=i*minLength j<=n-(k-i+1)*minLength
    枚举x
    x>=minLength, j-x>=(i-1)*minLength -> x<=j-(i-1)*minLength 
    j-x<=n-(k-i+1)*minLength -> x>=j+(k-i+1)*minLength-n
    s[j-x-1]是非质数 s[j-x]是质数
'''


class Solution:

    def beautifulPartitions(self, s: str, k: int, minLength: int) -> int:
        # dp[i][j] j个数字分成i段的方法数 i:[0,k] j:[0,n]
        # dp[i][j] = sum(dp[i-1][j-x]) s[j-x:j]符合要求
        # basecase dp[0][j] = 0 dp[0][0] = 1
        def is_prime(n):
            return n in '2357'

        mod = 10**9 + 7
        n = len(s)
        if n < k * minLength or not is_prime(s[0]) or is_prime(s[-1]):
            return 0

        def check(j):  # s[j]是否符合分割点
            return not is_prime(s[j]) and is_prime(s[j + 1])

        dp = [[0] * (n + 1) for _ in range(k + 1)]
        dp[0][0] = 1
        for i in range(1, k + 1):
            for j in range(i * minLength, n - (k - i) * minLength + 1):
                for x in range(max(j + (k - i + 1) * minLength - n, minLength), j - (i - 1) * minLength + 1):
                    idx = j - x - 1
                    if check(idx):
                        dp[i][j] += dp[i - 1][j - x]
                        dp[i][j] %= mod

        return dp[-1][-1] % mod


s = Solution()
s.beautifulPartitions("23542185131", 3, 3)


1

In [6]:
# 还不是很能理解
# https://leetcode.cn/problems/number-of-beautiful-partitions/solution/dong-tai-gui-hua-jian-ji-xie-fa-xun-huan-xyw3/
'''
1. 问题中有哪些变量?
分割个数 k
字符串长度 n

2. 重新复述一遍问题,替换变量名
把一个长度为j的字符串,分割出i段的合法方案数

3. (关键)最后一步发生了什么?
分割出一个子串
长度为 x ,且这个子串是s的一个后缀

4. 去掉最后一步,问题规模缩小了,变成什么样了?
把一个长度为 j-x 的字符串,分割出 i-1 段的合法方案数

5. 得到状态转移方程
2-> dp[i][j] 把一个长度为j的字符串,分割出i段的合法方案数
4-> dp[i][j] += dp[i-1][j'] j'是 i 段第一个字符的下标
    枚举j'
    j-j' >= minLength
    s[j']是质数 s[j'-1]是非质数
    
'''


class Solution:

    def beautifulPartitions(self, s: str, k: int, minLength: int) -> int:
        # dp[i][j] j个数字分成i段的方法数 i:[0,k] j:[0,n]
        # dp[i][j] = sum(dp[i-1][j-x]) s[j-x:j]符合要求
        # basecase dp[0][j] = 0 dp[0][0] = 1
        def is_prime(n):
            return n in '2357'

        mod = 10**9 + 7
        n = len(s)
        if n < k * minLength or not is_prime(s[0]) or is_prime(s[-1]):
            return 0

        def check(j):  # s[j]是否符合分割点
            return j == 0 or j == n or not is_prime(s[j - 1]) and is_prime(s[j])

        dp = [[0] * (n + 1) for _ in range(k + 1)]
        dp[0][0] = 1
        for i in range(1, k + 1):
            tot = 0
            # 优化：枚举的起点和终点需要给前后的子串预留出足够的长度
            for j in range(i * minLength, n - (k - i) * minLength + 1):
                # 绝了
                if check(j - minLength): # 右指针点符合就累加记录
                    tot = (tot + dp[i - 1][j - minLength]) % mod  # j'=j-minLength 双指针
                if check(j): # 右指针 满足就赋值
                    dp[i][j] = tot
        return dp[-1][-1] % mod


s = Solution()
s.beautifulPartitions("23542185131", 3, 3)


1